## Bayesian Binning into Quantiles (BBQ) Calibration Method, OVERVIEW

### Bayesian Binning into Quantiles is a calibration method, that does yada yada blah blah.

## 2.1 Load the model

We are going to use the pretrained ResNet18 Model from Torchvision for simplicity.

In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import torchvision
from torchvision.models import ResNet18_Weights
import torchvision.transforms as T
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

net = torchvision.models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
net.fc = nn.Linear(512, 10, device=device)
net = net.to(device)

Using device: cpu


## 2.2 Load the data

Here we are going to load the CIFAR10 dataset from torchvision to do a quick training session...

In [2]:
transforms = T.Compose([T.ToTensor()])

train = torchvision.datasets.CIFAR10(root="~/datasets", train=True, download=True, transform=transforms)
train, cal = torch.utils.data.random_split(train, [0.8, 0.2])
test = torchvision.datasets.CIFAR10(root="~/datasets", train=False, download=True, transform=transforms)
train_loader = DataLoader(train, batch_size=256, shuffle=True)
cal_loader = DataLoader(cal, batch_size=256, shuffle=True)
test_loader = DataLoader(test, batch_size=256, shuffle=False)

## 2.3 Train the model

So let's give the model a quick training session with our dataset.

In [3]:
epochs = 5
optimizer = optim.Adam(net.parameters())
criterion = nn.CrossEntropyLoss()
for epoch in tqdm(range(epochs)):
    net.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = net(inputs.to(device))
        loss = criterion(outputs, targets.to(device))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Running loss: {running_loss / len(train_loader)}")

  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [02:43<10:55, 163.91s/it]

Epoch 1, Running loss: 0.9217946464848367


 40%|████      | 2/5 [05:49<08:49, 176.38s/it]

Epoch 2, Running loss: 0.5628288703359616


...

Epoch 3, Running loss: 0.418066429484422


 80%|████████  | 4/5 [12:03<03:03, 183.51s/it]

Epoch 4, Running loss: 0.31996190842169864


100%|██████████| 5/5 [15:04<00:00, 180.98s/it]

Epoch 5, Running loss: 0.2449185317678816
